In [2]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [3]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)



In [3]:
chennai_zone = pd.read_csv('chennai_zone_new.csv')
chennai_hexes = chennai_zone['pickup_location_hex_8'].unique().tolist()
chennai_hexes.remove('88618c4f61fffff')
chennai_hexes.remove('88618c4f63fffff')
chennai_hexes.remove('88618c4d55fffff')
print(chennai_hexes)

['88618c4889fffff', '88618c4abbfffff', '88618c4f25fffff', '88618c48dbfffff', '88618c4899fffff', '88618c4d5dfffff', '88618c48d3fffff', '88618c489dfffff', '88618c4f07fffff', '88618c4f67fffff', '88618c4d4dfffff', '88618c4d49fffff', '88618c48d1fffff', '88618c4893fffff', '88618c48d7fffff', '88618c4ab1fffff', '88618c4f3dfffff', '88618c4d09fffff', '88618c4d47fffff', '88618c488bfffff', '88618c4f21fffff', '88618c4f05fffff', '88618c4f6dfffff', '88618c4f2bfffff', '88618c4f39fffff', '88618c48c1fffff', '88618c4891fffff', '88618c4f37fffff', '88618c48d5fffff', '88618c489bfffff', '88618c48c3fffff', '88618c48ddfffff', '88618c4f29fffff', '88618c4d45fffff', '88618c4f23fffff', '88618c4ab7fffff', '88618c4f27fffff', '88618c48d9fffff', '88618c4ab3fffff', '88618c4f33fffff', '88618c4f35fffff', '88618c4f65fffff', '88618c4d4bfffff', '88618c48cbfffff', '88618c4d41fffff', '88618c4f2dfffff', '88618c4d43fffff', '88618c4f31fffff', '88618c48c5fffff', '88618c48c7fffff']


In [5]:
print(len(chennai_hexes))

50


In [6]:
df_chennai = pd.DataFrame(chennai_hexes)
newdf = pd.DataFrame(np.repeat(df_chennai.values, 14, axis=0))
newdf.columns = ['pickup_location_hex_8']
newdf.head()

,pickup_location_hex_8
0,88618c4889fffff
1,88618c4889fffff
2,88618c4889fffff
3,88618c4889fffff
4,88618c4889fffff


In [7]:
newdf.to_csv('chennai_zone_hexes.csv', index = False)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230505'
and yyyymmdd <= '20230511'
and city_name in ('Chennai')
and service_obj_service_name = 'Link'
and hour in ('12', '13', '14', '15', '16')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

KeyboardInterrupt: 

In [ ]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,63fe0956c56010681e62b32e,1.802,88618c4f29fffff,13.040296,80.245430,Chennai,88618c48d3fffff,13.053395,88618c48d3fffff,13.053444,80.250315,88618c4f29fffff,13.040296,80.245430,80.250290,193158,19,1930,Link,19:59,2,20230228,dropped,Long Pickup
1,63fe0b21ae9613546e103bbf,1.523,88618c4183fffff,12.971894,80.254120,Chennai,88618c48d7fffff,13.060954,88618c48d7fffff,13.061236,80.261578,88618c4183fffff,12.971894,80.254120,80.261536,193937,19,1930,Link,19:59,2,20230228,dropped,None
2,63fe0d40085e0337bbbf1d18,2.376,88618c4f6dfffff,13.029403,80.263766,Chennai,88618c4f6dfffff,13.029425,88618c4f6dfffff,13.029531,80.263906,88618eb24bfffff,13.112084,80.252589,80.263824,194840,19,1945,Link,19:59,2,20230228,customerCancelled,Taking longer than expected
3,63fe0574085e0337bbbf0fc9,0.444,88618c4899fffff,13.062081,80.254395,Chennai,88618c4891fffff,13.068365,88618c4891fffff,13.067670,80.246329,88618c4899fffff,13.062081,80.254395,80.245491,191524,19,1915,Link,19:00,2,20230228,dropped,None
4,63fe062fc56010681e62adba,0.557,88618c4f35fffff,13.044477,80.225700,Chennai,88618c4f21fffff,13.049541,88618c4f21fffff,13.049525,80.245286,88618c4f35fffff,13.044477,80.225700,80.245270,191831,19,1915,Link,19:00,2,20230228,dropped,None


In [ ]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,63fe0956c56010681e62b32e,1.802,88618c4f29fffff,13.040296,80.245430,Chennai,88618c48d3fffff,13.053395,88618c48d3fffff,13.053444,80.250315,88618c4f29fffff,13.040296,80.245430,80.250290,193158,19,1930,Link,19:59,2,20230228,dropped,Long Pickup,58,31,28,1
1,63fe0b21ae9613546e103bbf,1.523,88618c4183fffff,12.971894,80.254120,Chennai,88618c48d7fffff,13.060954,88618c48d7fffff,13.061236,80.261578,88618c4183fffff,12.971894,80.254120,80.261536,193937,19,1930,Link,19:59,2,20230228,dropped,None,37,39,28,1
2,63fe0d40085e0337bbbf1d18,2.376,88618c4f6dfffff,13.029403,80.263766,Chennai,88618c4f6dfffff,13.029425,88618c4f6dfffff,13.029531,80.263906,88618eb24bfffff,13.112084,80.252589,80.263824,194840,19,1945,Link,19:59,2,20230228,customerCancelled,Taking longer than expected,40,48,28,1
3,63fe0574085e0337bbbf0fc9,0.444,88618c4899fffff,13.062081,80.254395,Chennai,88618c4891fffff,13.068365,88618c4891fffff,13.067670,80.246329,88618c4899fffff,13.062081,80.254395,80.245491,191524,19,1915,Link,19:00,2,20230228,dropped,None,24,15,28,1
4,63fe062fc56010681e62adba,0.557,88618c4f35fffff,13.044477,80.225700,Chennai,88618c4f21fffff,13.049541,88618c4f21fffff,13.049525,80.245286,88618c4f35fffff,13.044477,80.225700,80.245270,191831,19,1915,Link,19:00,2,20230228,dropped,None,31,18,28,1


In [4]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Delhi')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [6]:
df_cluster_copy[df_cluster_copy['hex_id'] == '883da11593fffff']

,city,cluster,hex_id,cluster_original,resolution,executiondate
3324,Delhi,Jamia Milia_Del,883da11593fffff,Jamia Milia_Del,8,2023-05-11


In [ ]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,63fe0956c56010681e62b32e,1.802,88618c4f29fffff,13.040296,80.245430,Chennai,88618c48d3fffff,13.053395,88618c48d3fffff,13.053444,80.250315,88618c4f29fffff,13.040296,80.245430,80.250290,193158,19,1930,Link,19:59,2,20230228,dropped,Long Pickup,58,31,28,1,Royapettah,Teynampet
1,63fe0b21ae9613546e103bbf,1.523,88618c4183fffff,12.971894,80.254120,Chennai,88618c48d7fffff,13.060954,88618c48d7fffff,13.061236,80.261578,88618c4183fffff,12.971894,80.254120,80.261536,193937,19,1930,Link,19:59,2,20230228,dropped,None,37,39,28,1,Thousand Lights Metro,Perungudi 2
2,63fe0d40085e0337bbbf1d18,2.376,88618c4f6dfffff,13.029403,80.263766,Chennai,88618c4f6dfffff,13.029425,88618c4f6dfffff,13.029531,80.263906,88618eb24bfffff,13.112084,80.252589,80.263824,194840,19,1945,Link,19:59,2,20230228,customerCancelled,Taking longer than expected,40,48,28,1,Mylapore,Perambur
3,63fe0574085e0337bbbf0fc9,0.444,88618c4899fffff,13.062081,80.254395,Chennai,88618c4891fffff,13.068365,88618c4891fffff,13.067670,80.246329,88618c4899fffff,13.062081,80.254395,80.245491,191524,19,1915,Link,19:00,2,20230228,dropped,None,24,15,28,1,Egmore,Nungambakkam
4,63fe062fc56010681e62adba,0.557,88618c4f35fffff,13.044477,80.225700,Chennai,88618c4f21fffff,13.049541,88618c4f21fffff,13.049525,80.245286,88618c4f35fffff,13.044477,80.225700,80.245270,191831,19,1915,Link,19:00,2,20230228,dropped,None,31,18,28,1,T Nagar,T Nagar


In [ ]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_cluster'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_cluster'])['order_id'].nunique().reset_index()
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,order_id
0,20230227,17,Arumbakkam 2,38
1,20230227,17,Ashok Nagar Chennai,35
2,20230227,17,Chepauk,57
3,20230227,17,Egmore,90
4,20230227,17,Mylapore,66


In [34]:
df_morning[df_morning['drop_location_hex_8'].isin(chennai_hexes)].shape[0]/df_morning.shape[0]

0.6392965204236006

In [28]:
df_morning.groupby(['yyyymmdd', 'hour'])['order_id'].nunique().reset_index()

,yyyymmdd,hour,order_id
0,20230227,08,1179
1,20230227,09,1677
2,20230227,10,1310
3,20230227,11,938
4,20230228,08,1024
5,20230228,09,1575
6,20230228,10,1180
7,20230228,11,878
8,20230301,08,1246
9,20230301,09,2104


In [29]:
#Query data for online captains

q_online_captains = """
select
        yyyymmdd, substr(hhmm, 1, 2) as hour, cluster,
        count(distinct captain_id) as Online_Captains
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd >= '20230227'
        and yyyymmdd <= '20230302'
        and city in ('Chennai')
        and a.resolution = '8' 
        and service in ('Link')
        and substr(hhmm, 1, 2) in ('08', '09', '10', '11', '17', '18', '19', '20', '21')
        and location in ('88618c4889fffff', '88618c4abbfffff', '88618c4f25fffff', '88618c48dbfffff', '88618c4899fffff', '88618c4d5dfffff', '88618c48d3fffff',
 '88618c489dfffff', '88618c4f07fffff', '88618c4f67fffff', '88618c4d4dfffff', '88618c4d49fffff', '88618c48d1fffff', '88618c4893fffff', '88618c48d7fffff',
  '88618c4ab1fffff', '88618c4f3dfffff', '88618c4d09fffff', '88618c4d47fffff', '88618c488bfffff', '88618c4f21fffff', '88618c4f05fffff', '88618c4f6dfffff',
   '88618c4f2bfffff', '88618c4f39fffff', '88618c48c1fffff', '88618c4891fffff', '88618c4f37fffff', '88618c48d5fffff', '88618c489bfffff', '88618c48c3fffff',
    '88618c48ddfffff', '88618c4f29fffff', '88618c4d45fffff', '88618c4f23fffff', '88618c4ab7fffff', '88618c4f27fffff', '88618c48d9fffff', '88618c4ab3fffff',
     '88618c4f33fffff', '88618c4f35fffff', '88618c4f65fffff', '88618c4d4bfffff', '88618c48cbfffff', '88618c4d41fffff', '88618c4f2dfffff', '88618c4d43fffff',
      '88618c4f31fffff', '88618c48c5fffff', '88618c48c7fffff')
)
group by 1,2,3
"""

#Load data into pandas table
df_online_captains_copy = pd.read_sql(q_online_captains, connection)

In [30]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.head()

,yyyymmdd,hour,cluster,Online_Captains
0,20230228,09,T Nagar,313
1,20230302,19,Vadapalani Thirunagar,162
2,20230227,20,Vadapalani East,126
3,20230227,20,T Nagar,640
4,20230227,18,Vadapalani,142


In [31]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230228,09,T Nagar,313
1,20230302,19,Vadapalani Thirunagar,162
2,20230227,20,Vadapalani East,126
3,20230227,20,T Nagar,640
4,20230227,18,Vadapalani,142


In [32]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
250,20230302,18,Thousand Lights Metro,92,127.0,0.7244
18,20230227,18,Egmore,139,199.0,0.6985
175,20230301,18,Thousand Lights Metro,87,126.0,0.6905
245,20230302,18,Nungambakkam,392,593.0,0.6610
243,20230302,18,Egmore,129,200.0,0.6450


In [19]:
df_evening_grouped.groupby(['hour'])['demand_to_supply_ratio'].mean().reset_index()

,hour,demand_to_supply_ratio
0,17,0.346028
1,18,0.391378
2,19,0.322843
3,20,0.249485
4,21,0.204045


In [33]:
df_morning_grouped.groupby(['hour'])['demand_to_supply_ratio'].mean().reset_index()

,hour,demand_to_supply_ratio
0,08,0.765935
1,09,0.887532
2,10,0.510102
3,11,0.324228


In [35]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
0,63febc47911e956e42049415,NaN,88618c4d41fffff,13.063969,80.220993,Chennai,88618c4d41fffff,13.063980,88618c4d41fffff,13.063955,80.220886,88618c4d0dfffff,13.074390,80.213921,80.220970,081527,08,0815,Link,08:00,3,20230301,customerCancelled,order cancelled before rider accepted,27,15,01,1,Nungambakkam,Anna Nagar,264,212.0,1.2453
1,63febe41911e956e420499e8,NaN,88618c4d5dfffff,13.055635,80.214500,Chennai,88618c4d5dfffff,13.055641,88618c4d5dfffff,13.055664,80.214104,88618c4d51fffff,13.058089,80.204460,80.214485,082353,08,0815,Link,08:00,3,20230301,customerCancelled,order cancelled before rider accepted,53,23,01,1,Vadapalani,Saligragam,54,56.0,0.9643
2,63febebe4516da6216be5682,2.433,88618c4883fffff,13.077768,80.261879,Chennai,88618c489dfffff,13.071818,88618c489dfffff,13.071760,80.253088,88618c4883fffff,13.077768,80.261879,80.253136,082558,08,0815,Link,08:00,3,20230301,dropped,Heavy Traffic,58,25,01,1,Egmore,Egmore Metro,40,94.0,0.4255
3,63febfaeeee8320e4c618e02,1.485,88618c4d4dfffff,13.065878,80.232613,Chennai,88618c4899fffff,13.060361,88618c4899fffff,13.060909,80.253366,88618c4d4dfffff,13.065878,80.232613,80.253220,082958,08,0815,Link,08:00,3,20230301,dropped,None,58,29,01,1,Nungambakkam,Nungambakkam,264,212.0,1.2453
4,63fec2c354cd0502c1c4b6b4,0.446,88618c48ddfffff,13.055532,80.270294,Chennai,88618c48ddfffff,13.055561,88618c48ddfffff,13.055359,80.270939,88618c48d5fffff,13.064194,80.270020,80.270309,084307,08,0830,Link,08:59,3,20230301,customerCancelled,Others,07,43,01,1,Chepauk,Royapettah,94,85.0,1.1059


In [36]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio,weekday_name
0,63febc47911e956e42049415,NaN,88618c4d41fffff,13.063969,80.220993,Chennai,88618c4d41fffff,13.063980,88618c4d41fffff,13.063955,80.220886,88618c4d0dfffff,13.074390,80.213921,80.220970,081527,08,0815,Link,08:00,3,20230301,customerCancelled,order cancelled before rider accepted,27,15,01,1,Nungambakkam,Anna Nagar,264,212.0,1.2453,Wednesday
1,63febe41911e956e420499e8,NaN,88618c4d5dfffff,13.055635,80.214500,Chennai,88618c4d5dfffff,13.055641,88618c4d5dfffff,13.055664,80.214104,88618c4d51fffff,13.058089,80.204460,80.214485,082353,08,0815,Link,08:00,3,20230301,customerCancelled,order cancelled before rider accepted,53,23,01,1,Vadapalani,Saligragam,54,56.0,0.9643,Wednesday
2,63febebe4516da6216be5682,2.433,88618c4883fffff,13.077768,80.261879,Chennai,88618c489dfffff,13.071818,88618c489dfffff,13.071760,80.253088,88618c4883fffff,13.077768,80.261879,80.253136,082558,08,0815,Link,08:00,3,20230301,dropped,Heavy Traffic,58,25,01,1,Egmore,Egmore Metro,40,94.0,0.4255,Wednesday
3,63febfaeeee8320e4c618e02,1.485,88618c4d4dfffff,13.065878,80.232613,Chennai,88618c4899fffff,13.060361,88618c4899fffff,13.060909,80.253366,88618c4d4dfffff,13.065878,80.232613,80.253220,082958,08,0815,Link,08:00,3,20230301,dropped,None,58,29,01,1,Nungambakkam,Nungambakkam,264,212.0,1.2453,Wednesday
4,63fec2c354cd0502c1c4b6b4,0.446,88618c48ddfffff,13.055532,80.270294,Chennai,88618c48ddfffff,13.055561,88618c48ddfffff,13.055359,80.270939,88618c48d5fffff,13.064194,80.270020,80.270309,084307,08,0830,Link,08:59,3,20230301,customerCancelled,Others,07,43,01,1,Chepauk,Royapettah,94,85.0,1.1059,Wednesday


In [37]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [38]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [39]:
df_morning_filter.to_csv('chennai_morning_zone.csv', index = False)
df_evening_filter.to_csv('chennai_evening_zone.csv', index = False)